## Fixes:
- sort in absolute value
- Put it in a module / pip / conda
- Add differential expression feature
- Calculate everything in matrices rather than for loops
- When you color cells, what should the max and min expression be? Based on the selected cells or based on all of the cells. Currently it seems to remap based on only the selected cells but this is maybe misleading.

- Pearson R vs. Spearman R?


- Required input values: Pandas vs. numpy


In [1]:
import pandas as pd
import numpy as np
from umap_explorer import UMAPexplorer


In [2]:
%load_ext py5
%gui osx

In [3]:
print("LOADING UMAP DATA...")

_umap = pd.read_csv('../data/pbmc3k_umap.tsv', sep='\t')
_umap.index = _umap['index']
_umap = _umap.iloc[:, 1:]
_umap.head()

LOADING UMAP DATA...


,UMAP_1,UMAP_2
index,,
AAACATACAACCAC-1,3.638991,3.122167
AAACATTGAGCTAC-1,-0.490064,10.553852
AAACATTGATCAGC-1,0.391078,4.035561
AAACCGTGCTTCCG-1,-6.747729,-0.600776
AAACCGTGTATGCG-1,1.887045,-1.526383


In [4]:
print("LOADING GENE EXPRESSION DATA...")

_expr = pd.read_csv('../data/pbmc3k_expression_filtered_normalized.tsv', sep='\t')
_expr.set_index('index', inplace=True)
_expr.head()

LOADING GENE EXPRESSION DATA...


,TNFRSF4,CPSF3L,ATAD3C,C1orf86,RER1,TNFRSF25,TNFRSF9,CTNNBIP1,SRM,UBIAD1,...,DSCR3,BRWD1,BACE2,SIK1,C21orf33,ICOSLG,SUMO3,SLC19A1,S100B,PRMT2
index,,,,,,,,,,,,,,,,,,,,,
AAACATACAACCAC-1,-0.171470,-0.280812,-0.046677,-0.475169,-0.544024,4.928496,-0.038028,-0.280573,-0.341788,-0.195361,...,-0.226570,-0.236269,-0.102943,-0.222116,-0.312401,-0.121678,-0.521229,-0.098269,-0.209095,-0.531203
AAACATTGAGCTAC-1,-0.214582,-0.372653,-0.054804,-0.683391,0.633951,-0.334837,-0.045589,-0.498264,-0.541914,-0.209017,...,-0.317531,2.568868,0.007155,-0.445372,1.629285,-0.058662,-0.857163,-0.266844,-0.313146,-0.596654
AAACATTGATCAGC-1,-0.376888,-0.295085,-0.057527,-0.520972,1.332648,-0.309362,-0.103108,-0.272526,-0.500799,-0.220228,...,-0.302938,-0.239801,-0.071774,-0.297857,-0.410920,-0.070431,-0.590720,-0.158656,-0.170876,1.379000
AAACCGTGCTTCCG-1,-0.285241,-0.281735,-0.052227,-0.484929,1.572680,-0.271825,-0.074552,-0.258875,-0.416752,-0.208471,...,-0.262978,-0.231807,-0.093818,-0.247770,2.552079,-0.097402,1.631684,-0.119462,-0.179120,-0.505669
AAACCGTGTATGCG-1,-0.256484,-0.220394,-0.046800,-0.345859,-0.333409,-0.208122,-0.069514,5.806440,-0.283112,-0.199355,...,-0.202237,-0.176765,-0.167350,-0.098665,-0.275836,-0.139482,-0.310095,-0.006877,-0.109614,-0.461946


In [5]:
test = UMAPexplorer(_umap, _expr)
test.explore_data()

Selected 33 cells
Calculating correlations...
Selected gene LGALS1
Min/max expression level for gene LGALS1 -1.772784948348999 1.889415979385376
EXPORTING DATA...
BYE


In [7]:
test.significant_genes.head()

,R,P
CDA,-19.105316,3.239770e-24
ASB8,-18.150373,5.595836e-37
TNFRSF25,-16.968708,1.782821e-50
DCAF8,-16.148441,5.402525e-44
PRR5,-15.726354,9.606228e-47


In [8]:
from scipy.stats import pearsonr, spearmanr, ttest_ind

In [9]:
selected = _expr.loc[test.selected_cells['index'], :]
remainder = _expr.loc[~_expr.index.isin(test.selected_cells['index']), :]

In [10]:
A = _expr.iloc[:,:10].values
B = _expr.iloc[:,11].values
print(A.shape, B.shape)

(2638, 10) (2638,)


In [19]:
res = ttest_ind(selected.values, remainder.values, axis=0, equal_var=False, nan_policy='propagate', alternative='two-sided')
res = pd.DataFrame([res.statistic, res.pvalue], columns=_expr.columns, index=['T', 'P']).T

In [20]:
res.loc[test.significant_genes.index, :].head()

,T,P
CDA,-19.105316,3.239770e-24
ASB8,-18.150373,5.595836e-37
TNFRSF25,-16.968708,1.782821e-50
DCAF8,-16.148441,5.402525e-44
PRR5,-15.726354,9.606228e-47


In [15]:
res = spearmanr(A,B.reshape(-1,1), axis=None)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 26380 and the array at index 1 has size 2638

In [16]:
A.shape

(2638, 10)

In [24]:
np.corrcoef(A, B)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 10 and the array at index 1 has size 2638

In [13]:
selected.shape

(33, 1838)

In [14]:
remainder.shape

(2605, 1838)

In [18]:
res.statistic

array([-4.35918297e+00,  2.72819241e-01, -4.04673577e+00, ...,
        1.71800118e-01, -1.31253541e+01, -1.50082618e-03])

In [19]:
res.pvalue

array([9.78690737e-05, 7.86695233e-01, 5.34310250e-05, ...,
       8.64664668e-01, 9.94317294e-37, 9.98811460e-01])

In [9]:
z = _umap.values.copy()

In [6]:
test.significant_genes

,R,P
LGALS1,-0.776084,4.668194e-07
LGALS3,-0.770768,6.255527e-07
AIF1,-0.760379,1.084620e-06
PCP2,-0.723926,6.138661e-06
URB2,-0.723828,6.165109e-06
...,...,...
RP11-432I5.1,0.723548,6.241083e-06
SPRY1,0.723828,6.165200e-06
CDKN2A,0.723898,6.146322e-06
RFPL2,0.723902,6.145310e-06


In [7]:
test.selected_cells

,index,proj
0,AAGATTACCGCCTT-1,0.254572
1,AAGCCATGAACTGC-1,0.525927
2,AATGCGTGGACGGA-1,0.118430
3,AATTACGAATTCCT-1,0.195944
4,ACGAGGGACAGGAG-1,0.658741
5,ACGTGATGCCATGA-1,0.330072
6,ACTTAAGATTACTC-1,0.096151
7,AGCACTGATGCTTT-1,0.424344
8,ATACCACTCTAAGC-1,0.184605
9,ATACCGGAATGCTG-1,0.418610


In [6]:
test = UMAPexplorer(_umap.values, _expr.values)
test.explore_data()

Selected 37 cells
Calculating correlations...
Done
Selected gene 1797
Min/max expression level for gene 1797 -0.979315221309662 2.0967912673950195
EXPORTING DATA...
BYE
